In [1]:
import pymongo
import operator
import itertools
from pprint import pprint

In [2]:
con = pymongo.MongoClient()
db = con.lacuerda
art = db.artists
ver = db.versions
songs = db.songs

def show_cursor(cursor, limit=10):
    for item in itertools.islice(cursor, limit):
        pprint(item)

### Ver si el album de una canción es el mismo en todas las versiones

In [ ]:
cur=ver.aggregate([
        {"$match": {"album": {"$ne": None}}},
        {"$group": {"_id": {"artista": "$artista", "slug": "$slug"},
                    # "versiones": {"$sum": 1},
                    "albums": {"$addToSet": "$album"}}},
        {"$project": {"_id": 1, "albums": 1, "album_count": {"$size": "$albums"}}},
        {"$sort": {"album_count": -1}},
        {"$limit": 10}
    ])
show_cursor(cur)

Conclusión: depende de la versión

### Ver si el compositor de una canción es el mismo en todas las versiones

In [ ]:
cur=ver.aggregate([
        {"$match": {"autor": {"$ne": None}}},
        {"$group": {"_id": {"artista": "$artista", "slug": "$slug"},
                    # "versiones": {"$sum": 1},
                    "autores": {"$addToSet": "$autor"}}},
        {"$project": {"_id": 1, "autores": 1, "autor_count": {"$size": "$autores"}}},
        {"$sort": {"autor_count": -1}},
        {"$limit": 10}
    ])
show_cursor(cur)

El artista también depende de la versión

### Búsqueda de texto

In [62]:
query = 'loco alta suciedad calamaro'
fields = {"versiones.acordes":0, "versiones.contenido": 0}
songs_search = db.songs_search
cur = songs_search.find({"$text": {"$search": query}},
                 #{"versiones.acordes":0, "versiones.contenido": 0, "score": {"$meta": "textScore"}}).sort(
                 {"_id": 1, "album": 1, "nombre_artista": 1, "nombre": 1, "score": {"$meta": "textScore"}}).sort(
    [('score', {"$meta": "textScore"})]).limit(5)
show_cursor(cur)

{'_id': 'andres_calamaro/alta_suciedad',
 'album': 'Alta Suciedad (1997)',
 'nombre': 'Alta Suciedad',
 'nombre_artista': 'Andrés Calamaro',
 'score': 8.833333333333332}
{'_id': 'andres_calamaro/loco',
 'album': 'Alta Suciedad (Ü¿¿)',
 'nombre': 'Loco',
 'nombre_artista': 'Andrés Calamaro',
 'score': 6.833333333333334}
{'_id': 'formula_v/loco_loco_casi_loco',
 'album': None,
 'nombre': 'Loco, loco casi loco',
 'nombre_artista': 'Formula V',
 'score': 6.125}
{'_id': 'chico_trujillo/loco_loco',
 'album': '2010 (2010)',
 'nombre': 'Loco Loco',
 'nombre_artista': 'Chico Trujillo',
 'score': 6.0}
{'_id': 'guffi/loco_loco',
 'album': None,
 'nombre': 'Loco, loco',
 'nombre_artista': 'Guffi',
 'score': 6.0}


In [56]:
query = 'no tan buenos aires andres calamaro honestidad brutal'
fields = {"versiones.acordes":0, "versiones.contenido": 0}
exp = songs.find({"$text": {"$search": query}},
                 {"versiones.acordes":0, "versiones.contenido": 0, "score": {"$meta": "textScore"}}).sort(
    [('score', {"$meta": "textScore"})]).limit(5).explain()
exp

{'executionStats': {'allPlansExecution': [],
  'executionStages': {'advanced': 5,
   'executionTimeMillisEstimate': 400,
   'inputStage': {'advanced': 5,
    'executionTimeMillisEstimate': 400,
    'inputStage': {'advanced': 1832,
     'docsExamined': 1832,
     'executionTimeMillisEstimate': 400,
     'indexName': 'nombre_text_nombre_artista_text_album_text',
     'indexPrefix': {},
     'invalidates': 0,
     'isEOF': 1,
     'keysExamined': 1975,
     'nReturned': 1832,
     'needFetch': 0,
     'needTime': 1983,
     'parsedTextQuery': {'negatedPhrases': [],
      'negatedTerms': [],
      'phrases': [],
      'terms': ['tan',
       'bueno',
       'air',
       'andr',
       'calamaro',
       'honestidad',
       'brutal']},
     'restoreState': 40,
     'saveState': 40,
     'stage': 'TEXT',
     'works': 3816},
    'invalidates': 0,
    'isEOF': 1,
    'limitAmount': 5,
    'memLimit': 33554432,
    'memUsage': 23226,
    'nReturned': 5,
    'needFetch': 0,
    'needTime': 38